In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines：使用Google Cloud Pipeline Components的AutoML图像分类管道

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_images.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial/pipelines%2Fgoogle_cloud_pipeline_components_automl_images.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td> 
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_images.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
     </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_images.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
  
</table>
<br/><br/>

## 概述

本笔记本展示了如何使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)中定义的组件，在[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)上构建一个AutoML图像分类工作流程。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[AutoML组件](https://cloud.google.com/vertex-ai/docs/pipelines/vertex-automl-component)的信息。

### 目标

在本教程中，您将学习如何使用Vertex AI Pipelines和Google Cloud管道组件构建AutoML图像分类模型。


本教程使用以下谷歌云ML服务：

- Vertex AI Pipelines
- Google Cloud管道组件
- Vertex AutoML
- Vertex AI模型资源
- Vertex AI端点资源

执行的步骤包括：

- 创建一个KFP管道：
    - 创建一个数据集资源。
    - 训练一个AutoML图像分类模型资源。
    - 创建一个端点资源。
    - 将模型资源部署到端点资源。
- 编译KFP管道。
- 使用Vertex AI Pipelines执行KFP管道。

这些组件在[此处记录](https://google-cloud-pipeline-components.readthedocs.io/en/latest/google_cloud_pipeline_components.aiplatform.html#module-google_cloud_pipeline_components.aiplatform)。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在公共云存储桶中。训练的模型可以预测出图像中的花卉类型，包括五种花卉：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 费用

此教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud 存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud 存储定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用量生成费用估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他所需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 kfp \
                                 google-cloud-pipeline-components==2.4.1 \
                                 google-cloud-storage

重启运行时（仅适用于Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。请等待直到它完成再继续下一步。 ⚠️</b>
</div>

### 在谷歌 Colab 上验证您的笔记本环境

在谷歌 Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

#### UUID

为了避免在创建资源时用户之间的名称冲突，为每个会话实例创建一个UUID。将这些UUID附加到本教程中创建的资源的相应名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 设置Google Cloud项目信息并初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type: "string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} {BUCKET_URI}

服务账号

**如果您不知道您的服务账号**，请尝试使用`gcloud`命令通过执行下面的第二个单元格获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置 Vertex AI 管道的服务帐号访问权限

运行以下命令，将您的服务帐号授权读取和写入管道中的工件，该工件存储在您在前一步中创建的存储桶中。您只需要针对每个服务帐号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

导入库并定义常量

In [ ]:
from typing import Any, Dict, List

import google.cloud.aiplatform as aip
import kfp
from kfp import compiler

#### Vertex AI管道常量

为Vertex AI管道设置以下常量：

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/flowers"

初始化用于您的项目和相应存储桶的Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 定义使用`google_cloud_pipeline_components`组件的AutoML图像分类模型流水线

接下来，您可以定义流水线。

创建并部署一个使用数据集资源的AutoML图像分类模型资源。

In [ ]:
@kfp.dsl.pipeline(name="automl-image-training-v2")
def pipeline(project: str = PROJECT_ID, region: str = LOCATION):
    from google_cloud_pipeline_components.v1.automl.training_job import \
        AutoMLImageTrainingJobRunOp
    from google_cloud_pipeline_components.v1.dataset import \
        ImageDatasetCreateOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    ds_op = ImageDatasetCreateOp(
        project=project,
        display_name="flowers",
        gcs_source="gs://cloud-samples-data/ai-platform/flowers/flowers.csv",
        import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
    )

    training_job_run_op = AutoMLImageTrainingJobRunOp(
        project=project,
        display_name="train-automl-flowers",
        prediction_type="classification",
        model_type="CLOUD",
        dataset=ds_op.outputs["dataset"],
        model_display_name="train-automl-flowers",
        training_fraction_split=0.6,
        validation_fraction_split=0.2,
        test_fraction_split=0.2,
        budget_milli_node_hours=8000,
    )

    endpoint_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name="train-automl-flowers",
    )

    ModelDeployOp(
        model=training_job_run_op.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        automatic_resources_min_replica_count=1,
        automatic_resources_max_replica_count=1,
    )

编译流水线。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="image_classification_pipeline.yaml"
)

In [ ]:
DISPLAY_NAME = "flowers_" + UUID

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="image_classification_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run()

! rm image_classification_pipeline.yaml

运行管道

接下来，运行管道。

点击生成的链接以在云控制台中查看您的运行。

在Google Cloud控制台中，许多管道DAG节点在单击它们时会展开或折叠。这是DAG的部分展开视图（单击图像以查看更大的版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_image_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_image_classif.png" width="40%"/></a>

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

### 从管道获取资源进行清理
#### 获取任务详情的功能

In [ ]:
def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
        if task_detail.task_name == task_name:
            return task_detail

In [ ]:
pipeline_task_details = (
    job.gca_resource.job_detail.task_details
)  # fetch pipeline task details


# fetch endpoint from pipeline and delete the endpoint
endpoint_task = get_task_detail(pipeline_task_details, "endpoint-create")
endpoint_resourceName = (
    endpoint_task.outputs["endpoint"].artifacts[0].metadata["resourceName"]
)
endpoint = aip.Endpoint(endpoint_resourceName)
# undeploy model from endpoint
endpoint.undeploy_all()
endpoint.delete()

# fetch model from pipeline and delete the model
model_task = get_task_detail(pipeline_task_details, "automl-image-training-job")
model_resourceName = model_task.outputs["model"].artifacts[0].metadata["resourceName"]
model = aip.Model(model_resourceName)
model.delete()


# fetch dataset from pipeline and delete the dataset
dataset_task = get_task_detail(pipeline_task_details, "image-dataset-create")
dataset_resourceName = (
    dataset_task.outputs["dataset"].artifacts[0].metadata["resourceName"]
)
dataset = aip.ImageDataset(dataset_resourceName)
dataset.delete()

# delete the pipelinejob
job.delete()

In [ ]:
delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI